# Trade Simulation
Models the Flow of Goods Around the World in Response to Central American Drift

In [12]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

## Data Preprocessing


In [6]:
trade_df = pd.read_csv("GTCDIT.csv")
trade_df.head()

,Destination_Label,Year,TransportMode_Label,Individual economies_Transport_expenditure_US_Value,Individual economies_Transport_expenditure_US_Footnote,Individual economies_Transport_expenditure_US_MissingValue,Individual economies_FOB_value_US_Value,Individual economies_FOB_value_US_Footnote,Individual economies_FOB_value_US_MissingValue,Individual economies_Kilograms_Value,...,Zambia_Kilograms_MissingValue,Zimbabwe_Transport_expenditure_US_Value,Zimbabwe_Transport_expenditure_US_Footnote,Zimbabwe_Transport_expenditure_US_MissingValue,Zimbabwe_FOB_value_US_Value,Zimbabwe_FOB_value_US_Footnote,Zimbabwe_FOB_value_US_MissingValue,Zimbabwe_Kilograms_Value,Zimbabwe_Kilograms_Footnote,Zimbabwe_Kilograms_MissingValue
0,Afghanistan,2018,Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2018,Sea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2018,Railway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2018,Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2019,Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# combine train and rode into a single mode called land
trade_df['TransportMode_Label'] = trade_df['TransportMode_Label'].replace({'Railway': 'Land', 'Road': 'Land'})

# delete columns whose names contain 'Footnote' or 'MissingValue'
cols_to_drop = [col for col in trade_df.columns if 'Footnote' in col or 'MissingValue' in col or 'Individual' in col]
trade_df.drop(columns=cols_to_drop, inplace=True)

# combine rows where Destination_Label, Year, and TransportMode_Label are the same, for numerical values if both NaN, leave NaN, otherwise sum Transport Expenditure, FOB value, and Kilograms
def custom_sum(series):
    if series.isna().all():
        return np.nan
    return series.sum(skipna=True)

# Define the key columns for grouping
key_columns = ['Destination_Label', 'Year', 'TransportMode_Label']

# Determine the numerical columns as all columns except the key columns.
numerical_columns = [col for col in trade_df.columns if col not in key_columns]


trade_df = trade_df.groupby(['Destination_Label', 'Year', 'TransportMode_Label'], as_index=False).agg(
    {col: custom_sum for col in numerical_columns}
)

trade_df.head()

,Destination_Label,Year,TransportMode_Label,Afghanistan_Transport_expenditure_US_Value,Afghanistan_FOB_value_US_Value,Afghanistan_Kilograms_Value,Albania_Transport_expenditure_US_Value,Albania_FOB_value_US_Value,Albania_Kilograms_Value,Algeria_Transport_expenditure_US_Value,...,Western Sahara_Kilograms_Value,Yemen_Transport_expenditure_US_Value,Yemen_FOB_value_US_Value,Yemen_Kilograms_Value,Zambia_Transport_expenditure_US_Value,Zambia_FOB_value_US_Value,Zambia_Kilograms_Value,Zimbabwe_Transport_expenditure_US_Value,Zimbabwe_FOB_value_US_Value,Zimbabwe_Kilograms_Value
0,Afghanistan,2018,Air,NaN,NaN,NaN,41.0,368.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2018,Land,NaN,NaN,NaN,10.0,95.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2018,Sea,NaN,NaN,NaN,8.0,18.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2019,Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2843.0,6637.0,185.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2019,Land,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2915.0,154401.0,14786.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
unique_countries = trade_df['Destination_Label'].unique()

world = gpd.read_file("natural_earth/ne_50m_admin_0_countries.shp")

countries_gdf = world[world['ADMIN'].isin(unique_countries)].copy()

# Compute the centroid of each country’s geometry.
countries_gdf['centroid'] = countries_gdf.geometry.centroid
countries_gdf['centroid_longitude'] = countries_gdf.centroid.x
countries_gdf['centroid_latitude'] = countries_gdf.centroid.y

AttributeError: The geopandas.dataset has been deprecated and was removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.

In [ ]:
# Flatten the dataset so that each row is a single edge on the graph by extracting the values for columns corresponding to a particular origin
# Features: Edge_Id, Origin_Label (Extracted from the Columns, matches set of Destination_Labels), Destination_Label, Year, Route (Longitude and Latitude of Points Along Trade Route), Distance, Duration, Central_America (Boolean: Route Interects with Central America), Volume (KG), Value, Cost (USD)

## Route Computation

In [ ]:
# As the crow flies 
def compute_air_route(long1, lat1, long2, lat2):
    return 0

def compute_sea_route():
    return 0

def compute_land_route(): 

In [ ]:
trade_df = pd.read_csv("US.TransportCosts_1107_20250415_205300.csv")
# Expected columns might include: 'origin', 'destination', 'trade_volume',
# 'air_share', 'land_share', 'sea_share', 'cost_per_shipment', etc.

# --- Step 2. Import the Drift File ---
# The drift file contains updated node (port/airport/land entry) coordinates 
# due to geographic drift. Its columns might include: 'node', 'new_lat', 'new_lon'.
drift_df = pd.read_csv("central_america_drift.csv")

# --- Step 3. Construct the Trade Network Graph ---
G = nx.DiGraph()

# Add nodes from the dataset. In practice, if you already have node coordinates
# (e.g., longitude and latitude), load them as defaults.
# If a node is in the drift file, update its coordinate accordingly.
all_nodes = pd.unique(trade_df[['origin', 'destination']].values.ravel())
for node in all_nodes:
    # Check if drift data exists for this node
    drift_info = drift_df[drift_df['node'] == node]
    if not drift_info.empty:
        lat = drift_info.iloc[0]['new_lat']
        lon = drift_info.iloc[0]['new_lon']
    else:
        # Fallback: use original coordinates if available or assign dummy values
        # (Consider adding a mapping or an external lookup for these values)
        lat, lon = np.nan, np.nan
    # Store the position as an attribute for later use (e.g., in plotting or distance computation)
    G.add_node(node, pos=(lon, lat))

# --- Step 4. Define Mode Cost Parameters and Add Edges ---
# Parameters for the cost function. These should be calibrated to your application.
alpha = {'land': 0.5, 'sea': 0.3, 'air': 1.0}   # variable cost per mile
beta  = {'land': 0.1, 'sea': 0.05, 'air': 0.2}   # time-based cost multiplier
gamma = {'land': 10,   'sea': 20,  'air': 30}     # fixed overhead fees
v     = {'land': 40,   'sea': 20,  'air': 500}    # average speeds (miles per hour)

# Loop over each trade route record to add edges.
# We'll implement basic eligibility: for example, the trade volume must exceed a threshold.
volume_threshold = 100  # adjust this threshold as needed

for idx, row in trade_df.iterrows():
    origin = row['origin']
    destination = row['destination']
    trade_volume = row['trade_volume']
    
    # Eligibility check: skip routes with low volume (or apply other filters)
    if trade_volume < volume_threshold:
        continue

    # Check that both nodes have been added (and have coordinate data)
    if origin not in G.nodes or destination not in G.nodes:
        continue

    # Retrieve positions (using longitude, latitude stored as (lon, lat))
    pos_origin = G.nodes[origin]['pos']
    pos_destination = G.nodes[destination]['pos']
    
    # Compute a (naïve) Euclidean distance:
    # Note: For real geospatial data, consider using geodesic calculations.
    if np.any(np.isnan(pos_origin)) or np.any(np.isnan(pos_destination)):
        continue  # Skip route if coordinates are missing

    distance = np.sqrt((pos_origin[0] - pos_destination[0])**2 + (pos_origin[1] - pos_destination[1])**2)
    
    # --- Determine the least-cost mode ---
    # Calculate cost per mode: C = alpha * d + beta * (d/v) + gamma + P_switch 
    # (Here we assume no mode-switch penalty, but you could add that later.)
    costs = {}
    for mode in ['land', 'sea', 'air']:
        costs[mode] = alpha[mode] * distance + beta[mode] * (distance / v[mode]) + gamma[mode]
    
    # Decide the mode with the minimum estimated cost:
    selected_mode = min(costs, key=costs.get)

    # --- Add the edge to the graph with the computed attributes ---
    G.add_edge(origin, destination,
               trade_volume=trade_volume,
               selected_mode=selected_mode,
               cost=costs[selected_mode],
               distance=distance,
               cost_breakdown=costs)

# --- Step 5. Dynamic Updates for Drift ---
# When drift changes node positions, update edge distances and recalc costs.
def update_edge_distances(graph):
    """Updates distances and cost calculations on all edges when node positions change."""
    for u, v, data in graph.edges(data=True):
        pos_u = graph.nodes[u]['pos']
        pos_v = graph.nodes[v]['pos']
        # Recompute Euclidean distance
        new_distance = np.sqrt((pos_u[0] - pos_v[0])**2 + (pos_u[1] - pos_v[1])**2)
        data['distance'] = new_distance
        mode = data.get('selected_mode')
        if mode:
            new_cost = alpha[mode] * new_distance + beta[mode] * (new_distance / v[mode]) + gamma[mode]
            data['cost'] = new_cost
            data['cost_breakdown'][mode] = new_cost

# Example: After you update nodes via drift, call:
# update_edge_distances(G)

# --- Edge Cases Considerations ---
# - **Missing or Incomplete Data:**  
#     If either the origin or destination has missing coordinates, skip that edge.
# - **Eligibility of a Trade Route:**  
#     The route might be ineligible if the trade_volume is below a threshold, or if modal shares (which can be added as further filters) fall below acceptable levels.
# - **Multiple Modes & Mode Switching:**  
#     In future work, you may need to consider cases where large shipments are split. This might involve
#     adjusting the cost model with capacity constraints or introducing mode-switch penalties.
# - **Dynamic Topology Changes:**  
#     If drift severs or significantly alters a route (for example, isolating a node), implement logic to remove or reassign edges.
    
# --- Step 6. Calculate the Least Cost Path ---
def calculate_least_cost_path(graph, source, target):
    """Uses Dijkstra's algorithm to compute the least cost path between two nodes."""
    try:
        path = nx.dijkstra_path(graph, source, target, weight='cost')
        total_cost = nx.dijkstra_path_length(graph, source, target, weight='cost')
        return path, total_cost
    except nx.NetworkXNoPath:
        print(f"No path found between {source} and {target}.")
        return None, None

# Example usage:
source_node = 'PortA'
target_node = 'PortB'
path, cost = calculate_least_cost_path(G, source_node, target_node)
print("Least cost path:", path, "with total cost:", cost)

# --- Step 7. Graph Visualization ---
def plot_graph(graph):
    """Plots the network with node positions."""
    pos = nx.get_node_attributes(graph, 'pos')
    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(graph, pos, node_size=300)
    nx.draw_networkx_edges(graph, pos, arrows=True)
    nx.draw_networkx_labels(graph, pos)
    plt.title("Trade Network Graph")
    plt.show()

plot_graph(G)
